# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

# NORVIG'S SOLUTION

In [5]:
import re
from collections import Counter


def words(text): return re.findall(r'\w+', text.lower())


WORDS = Counter(words(open('big.txt').read()))


def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N


def correction(sentence):
    "Most probable spelling correction for word."
    result = []
    for word in sentence:
        result.append(max(candidates(word), key=P))
    return result


def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


correction(['valuu'])


['value']

**Problem with Norvig's solution:**
It is not context sensitive.


In [27]:
from collections import Counter
import random

class CS_corrector:
    def __init__(self, ngram):
        self.ngram_file = ngram
        self.calc_words()
        self.conditional_probs = self.calc_conditional_probs()

    def calc_conditional_probs(self):
        '''
            To calculate the conditional probability from the ngram model
        '''
        conditional_probs = {}
        for two_words, count_w1_w2 in self.w1_w2_counter.items():
            w1, _ = two_words.split('_')
            conditional_probs[two_words] = int(
                count_w1_w2) / self.w1_counter[w1]
        return conditional_probs

    def calc_words(self):
        '''
            Collecting the information needed to build the ngram model
        '''
        self.words = []
        self.w1_w2_counter = {}
        self.w1_counter = {}
        with open(self.ngram_file, 'r', encoding='latin-1') as f:
            for line in f.readlines():
                count, w1, w2 = line.split()
                self.w1_w2_counter[w1+'_'+w2] = count
                if self.w1_counter.get(w1):
                    self.w1_counter[w1] += int(count)
                else:
                    self.w1_counter[w1] = int(count)
                self.words.append(w1)
                self.words.append(w2)
        self.words = Counter(self.words)

    def edit_distance(self, word1, word2):
        """Calculate the Levenshtein distance between two words."""
        if len(word1) < len(word2):
            return self.edit_distance(word2, word1)

        if not word2:
            return len(word1)

        previous_row = range(len(word2) + 1)

        for i, c1 in enumerate(word1):
            current_row = [i + 1]

            for j, c2 in enumerate(word2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))

            previous_row = current_row
        if previous_row[-1] != 0:
            return previous_row[-1]
        return 1

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): return (e2 for e1 in self.edits1(word)
                                    for e2 in self.edits1(e1))

    def known(self, words): return set(w for w in words if w in self.words)

    def candidates(self, word):
        '''
            Getting all the candidate words that can be the correction of word
        '''
        if len(word) == 1:
            return word
        l = []
        l.extend(self.known(self.edits1(word)))
        if len(word) > 4:
            l.extend(self.known(self.edits2(word)))
        l.append(word)
        return l

    def get_best_word(self, original_word, cands, next_word=None, previous_word=None):
        '''
            Choosing the best word from the candidates list
        '''
        best_word = None
        best_score = 0
        conditional_probs_w = 0.9
        frequency_w = 0.1
        edit_distance_w = 0.8

        if next_word or previous_word:
            for word in cands:
                gram_format = f'{word}_{next_word}' if next_word else f'{previous_word}_{word}'
                if self.conditional_probs.get(gram_format):
                    score = frequency_w * self.words[word]/len(self.words) + \
                        edit_distance_w * (1/self.edit_distance(word, original_word)) + \
                        conditional_probs_w * \
                        self.conditional_probs[gram_format]

                    if word == original_word and self.known([word]):
                        score += 0.1
                    if score > best_score:
                        best_score = score
                        best_word = word

        if best_word == None:
            # choose the most frequent candidate with the least edit distance
            for word in cands:
                if self.words[word] > 0:
                    score = 0.8 * self.words[word]/len(self.words) + \
                        0.2 * 1/(self.edit_distance(word, original_word))
                    if word == original_word and self.known([word]):
                        score += 0.1
                    if score > best_score:
                        best_word = word
                        best_score = score
        if best_word == None:
            return original_word
        return best_word

    def check(self, sentence):
        '''
            This function takes the sentence as an input and return the same sentence corrected
        '''
        result = []
        # correcting the first word
        first_word = sentence[0]
        if len(sentence) > 1:
            if self.words[first_word] <= 50:  # mean that this word need correction
                next_word = sentence[1]
                cands = self.candidates(first_word)
                best_word = self.get_best_word(first_word, cands, next_word)
                result.append(best_word)
            else:
                result.append(first_word)
        else:
            cands = self.candidates(first_word)
            return self.get_best_word(first_word, cands)

        # correcting the rest of the sentence
        for i in range(1, len(sentence)):
            # if the word frequency is less than 50, then it is probably misspelled
            if self.words[sentence[i]] <= 50:
                cands = self.candidates(sentence[i])
                previous_word = result[i-1]
                best_word = self.get_best_word(
                    sentence[i], cands, previous_word=previous_word)
                result.append(best_word)
            else:
                result.append(sentence[i])
        return result

In [28]:
my_corrector = CS_corrector('bigrams.txt')
print(my_corrector.check(["English", "is", "a", "dming", "language"]))
print(my_corrector.check(["The", "men", "are", "dming", "sports"]))

['english', 'is', 'a', 'dying', 'language']
['the', 'men', 'are', 'doing', 'sports']


As we can clearly see, our program can grasp the context and answer correctly.

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

1. Bigrams are chosen for spell correction because they provide a balance between capturing contextual information and managing computational complexity. They offer reasonable accuracy while requiring less data and computational resources compared to higher-order n-grams.

2. Edit Distance Weights: Weights are assigned to factors like frequency, edit distance, and conditional probability to balance their influence on correction decisions. Instead of running long epochs, we brute forced weights due to limited time for the deadline.

3. Candidate Selection: Candidate words are generated based on common edit operations and favor known words from the dataset.

4. Scoring and Word Selection: Words are scored based on multiple factors, including contextual information, to determine the best correction.
 

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

***I could not create a test set and evaluate, due to limited deadlines.***